In [2]:
import subprocess
import regex as re
import sys
from collections import defaultdict

knumbers_unfiltered = ["K" + str((5-len(str(i)))*"0") + str(i) for i in range(27110)]

def concat_clusters(group_no, knumbers):
    knumbers_range = knumbers[(group_no - 1) * 100: group_no * 100]
    paths = ["/data/luojaa/kegg/kog_uid/{}.tsv".format(knumber) for knumber in knumbers_range]
    outfilename = "/data/luojaa/kegg/uid_groups/group{}.tsv".format(group_no)
    logfilename = "/data/luojaa/kegg/uidsdisordered.log"
    with open(outfilename, "a") as outfile, open(logfilename, "a") as logfile:
        for path in paths:
            try:
                with open(path, "r") as f:
                    uidprev = "00000"
                    lines = f.readlines()
                    for line in lines:
                        uid = line.strip().split("\t")[1]
                        if uid < uidprev:
                            print(group_no, file=logfile)
                        print(uid, file=outfile)
                        uid = uidprev
            except:
                continue
def assign_batches(group_no):
    batch_ids = defaultdict(list)
    batch_id, ind = 1, 1
    infilename = "/data/luojaa/kegg/uid_groups/group{}.tsv".format(group_no)
    with open(infilename, "r") as f:
        lines = f.readlines()
        for line in lines:
            if ind % 10000 == 0:
                batch_id += 1
            batch_ids[batch_id] += [line.strip()]
            ind += 1
    return batch_ids
def run_batch(ids):
    requesturl = "https://rest.uniprot.org/idmapping/run"
    from_db = "\'from=\"UniProtKB_AC-ID\"\'"
    to_db = "\'to=\"UniRef100\"\'"
    idstr = "\'ids=\"" + ",".join(ids) + "\"\'"
    command = "curl --request POST {} --form {} --form {} --form {}".format(requesturl, from_db, to_db, idstr)
    response = subprocess.run(command, text=True, stdout=subprocess.PIPE, check=True, shell=True)
    print(command)
    dict_str_out = response.stdout # str:{"jobId":"62910de2cb548388eac8e873394b006e9c2c16de"}
    print(dict_str_out)
    jobid = re.findall(r"\{\"jobId\"\:\"(.+)\"\}", dict_str_out)[0]
    return jobid

def run_batches(groups):
    for group_no in groups:
        #concat_clusters(group_no, knumbers_unfiltered)
        batches = assign_batches(group_no)
        with open("/data/luojaa/kegg/batch_slowjobids.tsv", "a") as outfile:
            for batch in batches.keys():
                uids = batches[batch]
                jobid = run_batch(uids)
                batch_id = "{}.{}".format(group_no, batch)
                print("\t".join([batch_id, jobid]), file=outfile)
                
#start_group, end_group = 19, 20
#groups = [i for i in range(start_group, end_group)]
#run_batches(groups)





In [3]:
jobid_file = f"/data/luojaa/kegg/batch_slowjobids.tsv"
def check_jobstatus(jobid):
    command = f"curl -i 'https://rest.uniprot.org/idmapping/status/{jobid}'"
    response = subprocess.run(
        command,
        text=True,
        stdout=subprocess.PIPE,
        check=True,
        shell = True)
    return response.stdout

with open(jobid_file, "r") as f:
    lines = f.readlines()
    for line in lines:
        batch, jobid = line.strip().split("\t")
        if float(batch) > 19:
            status_out = check_jobstatus(jobid)
            status = re.findall(r"jobStatus\":\"(.*)\"}", status_out)[0]
            print(status)
            if status != "FINISHED":
                print(batch)

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100    24    0    24    0     0     36      0 --:--:-- --:--:-- --:--:--    36
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0

FINISHED


100    24    0    24    0     0     61      0 --:--:-- --:--:-- --:--:--    61
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0

FINISHED


100    19    0    19    0     0     50      0 --:--:-- --:--:-- --:--:--    50
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0

...

100    23    0    23    0     0     61      0 --:--:-- --:--:-- --:--:--    61
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0

RUNNING
19.21


100    19    0    19    0     0     51      0 --:--:-- --:--:-- --:--:--    51
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0

NEW
19.22
FINISHED


100    24    0    24    0     0     63      0 --:--:-- --:--:-- --:--:--    63


In [14]:
#run_batches(groups)